In [29]:
# generate data first

In [30]:
#!ipython dataloader.py

In [31]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from pandas.api.types import CategoricalDtype
from pandas_profiling import ProfileReport

## Run the cell below first 

* Use add datepart from fastai: do this and switch kernel to something else.
* run with python version 3.10.6 and fast.ai 2.7.6
* then switch back to python 3.9.12

In [32]:
# from fastai.tabular.all import *
# df = pd.read_csv("Insurance_data_ke.csv", parse_dates=["Date_of_entry"])
# df = add_datepart(df, field_name="Date_of_entry")
# df.to_csv("Insurance_data_ke_featureeng.csv", index=False)

In [33]:
# set working directory to current directory
os.chdir("/home/bens/Desktop/data_privacy_011/data/")

In [34]:
#!cat Insurance_data_ke.txt

In [35]:
# Specify Dtypes: This speeds up loading data into memory and saves memory too :)
# try standardize int64 to int32
dtypes = {"County": object, "Gender": CategoricalDtype(), 
          "Martial_status": "category", "No_of_dependents": np.int32, "Age": np.int32, 
          "Tier": CategoricalDtype(categories=["Bronze","Silver","Gold"], ordered=True), 
          "Policy_limit": np.float32, "Illness_category": "category", "Updated_subscription": object, 
          "Updated_subscription": object, "Account_withdrawal": object, "Retention": np.int32, 
          "ID_number": np.int32, "Tax_id": object}

In [36]:
!ls ../data

feature_engineered_insurance2.csv  Insurance_data_ke.csv
feature_engineered_insurance.csv   Insurance_data_ke_featureeng.csv
Insurance_data_ke_001.csv	   Organs.csv


In [37]:
os.getcwd()

'/home/bens/Desktop/data_privacy_011/data'

In [38]:
# load datest
df = pd.read_csv("Insurance_data_ke_featureeng.csv", index_col="id", dtype=dtypes)
df2 = pd.read_csv("Insurance_data_ke_featureeng.csv", index_col="id", parse_dates=True)

In [39]:
# see DataFrame summaries
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   County                         1001 non-null   object  
 1   Gender                         1001 non-null   category
 2   Marital_status                 1001 non-null   object  
 3   No_of_dependents               1001 non-null   int32   
 4   Age                            1001 non-null   int32   
 5   Tier                           1001 non-null   category
 6   Policy_limit                   1001 non-null   float32 
 7   Illness_category               1001 non-null   category
 8   Updated_subscription           1001 non-null   object  
 9   Account_withdrawal             1001 non-null   object  
 10  Retention                      1001 non-null   int32   
 11  ID_number                      1001 non-null   int32   
 12  Tax_id                         100

In [40]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   County                         1001 non-null   object 
 1   Gender                         1001 non-null   object 
 2   Marital_status                 1001 non-null   object 
 3   No_of_dependents               1001 non-null   int64  
 4   Age                            1001 non-null   int64  
 5   Tier                           1001 non-null   object 
 6   Policy_limit                   1001 non-null   int64  
 7   Illness_category               1001 non-null   object 
 8   Updated_subscription           1001 non-null   object 
 9   Account_withdrawal             1001 non-null   object 
 10  Retention                      1001 non-null   int64  
 11  ID_number                      1001 non-null   int64  
 12  Tax_id                         1001 non-null   o

# Proved it :P

In [41]:
# confirm if the column is indeed a category
df.Tier

id
0       Silver
1       Silver
2       Bronze
3         Gold
4       Silver
         ...  
996     Silver
997     Bronze
998     Bronze
999       Gold
1000    Silver
Name: Tier, Length: 1001, dtype: category
Categories (3, object): ['Bronze' < 'Silver' < 'Gold']

In [42]:
# EDA summarized with Pandas profiling
# profile = ProfileReport(df)

# profile

In [43]:
# make a copy of the dataset 
df_copy = df

In [44]:
df.columns

Index(['County', 'Gender', 'Marital_status', 'No_of_dependents', 'Age', 'Tier',
       'Policy_limit', 'Illness_category', 'Updated_subscription',
       'Account_withdrawal', 'Retention', 'ID_number', 'Tax_id',
       'Date_of_entryYear', 'Date_of_entryMonth', 'Date_of_entryWeek',
       'Date_of_entryDay', 'Date_of_entryDayofweek', 'Date_of_entryDayofyear',
       'Date_of_entryIs_month_end', 'Date_of_entryIs_month_start',
       'Date_of_entryIs_quarter_end', 'Date_of_entryIs_quarter_start',
       'Date_of_entryIs_year_end', 'Date_of_entryIs_year_start',
       'Date_of_entryElapsed'],
      dtype='object')

In [45]:
# drop columns
df = df.drop(["ID_number", "Tax_id"], axis=1)

In [46]:
# make a distinction between columns
object_columns = df.dtypes == "object"
numeric_columns = df.dtypes == "float32"
category_columns = df.dtypes == "category"

In [47]:
# Make a boolean mask for categorical columns
cat_mask_obj = (df.dtypes == "object") | (df.dtypes == "category") | (df.dtypes == "bool")

# Get list of categorical column names
cat_mask_object = df.columns[cat_mask_obj].tolist()

# now for numerical columns
# anything that was parsed as float64 is numeric: make a boolean mask for that
cat_mask_numeric = (df.dtypes == "float32") | (df.dtypes == "int32") | (df.dtypes == "int64") | (df.dtypes == "float64")
cat_mask_numeric = df.columns[cat_mask_numeric].tolist()

# see the result in a combined list: to the left categorical and the right we have numeric columns
print(cat_mask_object, "\n \n", cat_mask_numeric)

['County', 'Gender', 'Marital_status', 'Tier', 'Illness_category', 'Updated_subscription', 'Account_withdrawal', 'Date_of_entryIs_month_end', 'Date_of_entryIs_month_start', 'Date_of_entryIs_quarter_end', 'Date_of_entryIs_quarter_start', 'Date_of_entryIs_year_end', 'Date_of_entryIs_year_start'] 
 
 ['No_of_dependents', 'Age', 'Policy_limit', 'Retention', 'Date_of_entryYear', 'Date_of_entryMonth', 'Date_of_entryWeek', 'Date_of_entryDay', 'Date_of_entryDayofweek', 'Date_of_entryDayofyear', 'Date_of_entryElapsed']


In [48]:
# conversion of our dataframe into a dictionary so as to use DictVectorizer
# this function is mostly used in text processing
df_dict = df[cat_mask_object].to_dict("records")

# Make a DictVectorizer: use documentation to learn how it works
# In short, it speeds up one hot encoding with meaningful columns created
# we don't want a sparse matrix right?
dv = DictVectorizer(sparse = False)

# Apply fit_transform to our dataset
df_encoded = dv.fit_transform(df_dict)

# see 10 rows
print (df_encoded[:10,:])
print ("=" * 100) # just formatting to distinguish outputs

# print the vocabulary that is, the columns of the dataset, note that order changes
# upon transformation
print(dv.vocabulary_)
print ("=" * 100) # more formatting

print(df_encoded.shape) # number of rows and columns for the encoded dataset
print(df[cat_mask_object].shape) # number of rows and columns for the original dataset
print("After doing the transformation the columns increase to 21.")

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [54]:
# simply taking the vectorized columns and the numeric columns and bringing them together
# to make an array for a classifier
concat_cols = np.hstack((df_encoded, df[cat_mask_numeric].values))

# another version that is in dataframe format
# make a dataframe with the encoded features and give the columns names from the dictVectorizer
df_cat_var = pd.DataFrame(df_encoded, columns=dv.get_feature_names_out())

# combine the columns together with the categorical features i.e add columns to the numerical dataframe with other dataframe with the categorical and object data types
concat_cols_df = pd.concat([df[cat_mask_numeric], df_cat_var, df.reset_index()[['id']]], axis=1)
concat_cols.shape

(1001, 80)

In [55]:
concat_cols_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
No_of_dependents,2.0,3.0,3.0,4.0,5.0,0.0,1.0,3.0,3.0,2.0,...,5.0,2.0,3.0,3.0,0.0,0.0,4.0,3.0,5.0,0.0
Age,47.0,39.0,73.0,52.0,39.0,34.0,42.0,46.0,43.0,31.0,...,45.0,68.0,65.0,41.0,59.0,64.0,41.0,53.0,67.0,61.0
Policy_limit,300000.0,300000.0,500000.0,300000.0,300000.0,500000.0,300000.0,1000000.0,500000.0,300000.0,...,300000.0,500000.0,500000.0,300000.0,1000000.0,300000.0,500000.0,300000.0,300000.0,300000.0
Retention,14.0,5.0,2.0,19.0,9.0,4.0,13.0,14.0,8.0,18.0,...,3.0,7.0,19.0,19.0,9.0,13.0,17.0,15.0,8.0,11.0
Date_of_entryYear,2013.0,2015.0,2000.0,2006.0,2016.0,2012.0,2005.0,2009.0,2019.0,2001.0,...,2001.0,2009.0,2019.0,2013.0,2006.0,2006.0,2016.0,2002.0,2019.0,2002.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tier=Gold,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Tier=Silver,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
Updated_subscription=No,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
Updated_subscription=Yes,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


In [56]:
# checking types of all columns
concat_cols_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1001 entries, 0 to 1000
Data columns (total 81 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   No_of_dependents                       1001 non-null   int32  
 1   Age                                    1001 non-null   int32  
 2   Policy_limit                           1001 non-null   float32
 3   Retention                              1001 non-null   int32  
 4   Date_of_entryYear                      1001 non-null   int64  
 5   Date_of_entryMonth                     1001 non-null   int64  
 6   Date_of_entryWeek                      1001 non-null   int64  
 7   Date_of_entryDay                       1001 non-null   int64  
 8   Date_of_entryDayofweek                 1001 non-null   int64  
 9   Date_of_entryDayofyear                 1001 non-null   int64  
 10  Date_of_entryElapsed                   1001 non-null   float64
 11  Acco

In [57]:
concat_cols_df.to_csv("feature_engineered_insurance2.csv",index=False)